In [ ]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import TextLoader, WebBaseLoader, PyPDFLoader, DirectoryLoader, UnstructuredMarkdownLoader
# import bs4
# import os

# # Document loading
# loader = DirectoryLoader(
#     path="dd",  # Replace with your folder path
#     loader_cls=UnstructuredMarkdownLoader,
#     use_multithreading=True
# )

# txt = loader.load()
# from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# from langchain_community.vectorstores import FAISS
# from pydantic.v1 import BaseModel, Field
# import os

# # Add this before your FAISS imports
# class VectorStoreConfig(BaseModel):
#     class Config:
#         arbitrary_types_allowed = True


# # Text splitting
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200,
#     separators=[
#         # ----- Markdown -----
#         r"\n# ", r"\n## ", r"\n### ", r"\n#### ",
#         r"\n```\n",
#         r"\n\*\*\*\n", r"\n---\n",

#         # ----- LaTeX/Math -----
#         r"\n\\begin{equation}",
#         r"\n\\begin{align}",
#         r"\n\\section{", r"\n\\subsection{",
#         r"\n\$\$", r"\n\\\[",

#         # ----- C Code -----
#         r"\n}\n",
#         r"\n// -{4,}\n",
#         r"\n#ifdef ", r"\n#endif",
#         r"\n}\n\n",

#         # ----- Assembly -----
#         r"\n\.section\s",
#         r"\n\.global\s",
#         r"\n[a-zA-Z_]+:\n",
#         r"\n; -{4,}\n",

#         # ----- Generic Fallbacks -----
#         r"\n\n", r"\n", r" ", r""
#     ],
#     is_separator_regex=True,
#     keep_separator=True,
#     strip_whitespace=True,
# )
# fdocs = text_splitter.split_documents(txt)

# # Embedding with CPU optimization
# embeddings = SentenceTransformerEmbeddings(
#     model_name="all-MiniLM-L12-v2",  
#     model_kwargs={"device": "cpu"},
#     encode_kwargs={
#         "batch_size": 64,
#         "normalize_embeddings": True
#     }
# )

# # Create FAISS index
# db = FAISS.from_documents(
#     documents=fdocs,
#     embedding=embeddings,
#     normalize_L2=True
# )

# # Save the FAISS index to disk
# save_directory = "./faiss_index"
# os.makedirs(save_directory, exist_ok=True)
# db.save_local(save_directory)


OSError: [WinError 127] La procédure spécifiée est introuvable. Error loading "c:\Users\PC\anaconda3\envs\mistral-finetune\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

: 

In [1]:
!pip install google-generativeai pytesseract Pillow pdf2image

  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
  Using cached grpcio_status-1.71.0-py3-none-any.whl.metadata (1.1 kB)
Using cached google_generativeai-0.8.5-py3-none-any.whl (155 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 

In [3]:
import pathlib
import os
import pytesseract
from PIL import Image # Needed by pytesseract
from pdf2image import convert_from_path # To convert PDF pages to images
# Removed google.generativeai and time imports

# --- Configuration ---
# 🚨 Tesseract Path (MODIFY IF NEEDED) ---
# If Tesseract is not in your system PATH, uncomment and set the correct path below
# On Windows, it might look like: r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# On Linux/macOS, it might be '/usr/bin/tesseract' or '/usr/local/bin/tesseract'
# pytesseract.pytesseract.tesseract_cmd = r'/path/to/your/tesseract'

# 🚨 Poppler Path (MODIFY IF NEEDED) ---
# If Poppler is not in your system PATH, provide the path to its 'bin' directory
# Example for Windows: POPPLER_PATH = r"C:\path\to\poppler-23.08.0\Library\bin"
# Example for macOS/Linux (if installed elsewhere): POPPLER_PATH = "/opt/homebrew/opt/poppler/bin"
POPPLER_PATH = None # Set to None to rely on PATH, or specify the path string

# --- Constants ---
INPUT_PDF_FOLDER = 'COURS ALSDDD/LANGAGE Z'  # Folder containing input PDFs
OUTPUT_MD_FOLDER = 'OUTPUT_MD_ALSDD_Zlanguage_OCR_RAW'  # Folder to save output Markdown files
TESSERACT_LANG = 'fra' # Specify French language for Tesseract OCR

# --- Ensure Output Directory Exists ---
pathlib.Path(OUTPUT_MD_FOLDER).mkdir(parents=True, exist_ok=True)

print(f"Starting OCR processing for PDFs in folder: '{INPUT_PDF_FOLDER}'")
print(f"Raw Markdown output will be saved in: '{OUTPUT_MD_FOLDER}'")
print(f"Using Tesseract OCR Language: '{TESSERACT_LANG}'")
print("-" * 30)

# --- Processing Loop ---
pdf_files = [f for f in os.listdir(INPUT_PDF_FOLDER) if f.lower().endswith('.pdf')]

if not pdf_files:
    print(f"No PDF files found in '{INPUT_PDF_FOLDER}'. Exiting.")
else:
    for filename in pdf_files:
        input_filepath = pathlib.Path(INPUT_PDF_FOLDER) / filename
        # Create output filename with .md extension
        if filename.lower().endswith('.pdf'):
            output_filename = filename[:-4] + '.md'
        else:
            output_filename = filename + '.md'
        output_filepath = pathlib.Path(OUTPUT_MD_FOLDER) / output_filename

        print(f"Processing: '{filename}'...")

        extracted_text = ""
        try:
            # 1. Convert PDF to Images
            print(f"  Converting PDF to images...")
            try:
                 # Pass poppler_path if it's set, otherwise pdf2image tries PATH
                 images = convert_from_path(input_filepath, poppler_path=POPPLER_PATH)
            except Exception as convert_err: # Catch specific pdf2image errors if known
                 print(f"  ❌ ERROR: Failed to convert PDF '{filename}' to images.")
                 print(f"      Check if Poppler is installed and accessible via PATH or if POPPLER_PATH is set correctly.")
                 print(f"      Error details: {convert_err}")
                 continue # Skip to next file

            if not images:
                print(f"  ⚠️ WARNING: No images were generated from PDF '{filename}'. It might be empty or corrupted.")
                continue # Skip to next file

            # 2. Perform OCR on each image using Tesseract (French)
            print(f"  Performing OCR with Tesseract (lang='{TESSERACT_LANG}')...")
            full_text_parts = []
            ocr_success = True
            for i, image in enumerate(images):
                try:
                    # Use Tesseract to do OCR on the image with French language
                    page_text = pytesseract.image_to_string(image, lang=TESSERACT_LANG)
                    full_text_parts.append(page_text)
                    print(f"    Processed page {i+1}/{len(images)}")
                except pytesseract.TesseractNotFoundError:
                    print(f"  ❌ CRITICAL ERROR: Tesseract executable not found. Please install Tesseract, add it to PATH, or set pytesseract.tesseract_cmd.")
                    ocr_success = False # Signal critical failure
                    break # Stop processing pages for this file
                except Exception as ocr_err:
                    print(f"  ⚠️ WARNING: OCR failed for page {i+1} of {filename}: {ocr_err}")
                    full_text_parts.append(f"\n\n--- OCR Error on Page {i+1} ---\n\n") # Add error marker

            if not ocr_success:
                 continue # Skip saving for this file due to Tesseract not found

            # Join pages with a clear separator
            extracted_text = "\n\n------ Page Break ------\n\n".join(full_text_parts)

            if not extracted_text.strip():
                 print(f"  ⚠️ WARNING: OCR extracted no text from '{filename}'. Output file will be empty or contain only error markers.")
                 # Still create the file, but it might be empty

            # 3. Save Raw Extracted Text Directly to Markdown File
            print(f"  Saving raw OCR text to Markdown file...")

            # Create simple Markdown content with a header and the raw text in a code block
            output_content = f"# Raw OCR Text from: {filename}\n\n"
            output_content += f"*(Language: {TESSERACT_LANG})*\n\n"
            output_content += "```text\n" # Use a text code block for the raw output
            output_content += extracted_text if extracted_text.strip() else "[No text extracted or only errors occurred]"
            output_content += "\n```\n"

            with open(output_filepath, 'w', encoding='utf-8') as f:
                f.write(output_content)

            print(f"    Successfully extracted text. Raw OCR output saved to: '{output_filepath}'")

        # --- Error Handling ---
        except FileNotFoundError:
            print(f"  ❌ ERROR: Input file not found: '{input_filepath}'")
        except pytesseract.TesseractNotFoundError:
             # This should be caught earlier, but as a safety net
             print(f"  ❌ CRITICAL ERROR: Tesseract is not installed or configured correctly for file '{filename}'. Skipping.")
        except Exception as e:
            # Catch other potential errors (permissions, unexpected Pillow/pdf2image issues)
            print(f"  ❌ UNEXPECTED ERROR processing '{filename}': {type(e).__name__} - {e}")

        print("-" * 30)

    print("Processing finished.")

Starting OCR processing for PDFs in folder: 'COURS ALSDDD/LANGAGE Z'
Raw Markdown output will be saved in: 'OUTPUT_MD_ALSDD_Zlanguage_OCR_RAW'
Using Tesseract OCR Language: 'fra'
------------------------------


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'COURS ALSDDD/LANGAGE Z'

In [2]:
!pip install unstructured

  Using cached unstructured-0.17.2-py3-none-any.whl.metadata (24 kB)
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
  Using cached python_iso639-2025.2.18-py3-none-any.whl.metadata (14 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached unstructured_client-0.34.0-py3-none-any.whl.metadata (21 kB)
  Using cached python_oxmsg-0.0.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached olefile-0.47-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached cryptography-44.0.3-cp39-abi3-win_amd64.whl.metadata (5.7 kB)
  Using cached eva

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
morphik 0.1.4 requires pydantic==2.10.3, but you have pydantic 2.11.4 which is incompatible.
pix2tex 0.1.4 requires timm==0.5.4, but you have timm 1.0.15 which is incompatible.


In [ ]:

# To load it back later:
# loaded_db = FAISS.load_local(save_directory, embeddings)

# Now let's modify to use GROQ API instead of Ollama
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq  # Import the GROQ chat model
from langchain_core.prompts import ChatPromptTemplate
import os

# Initialize the GROQ LLM - replaced Ollama with GROQ
llm = ChatGroq(
    
    model='llama-3.3-70b-versatile',
    temperature=0,
    max_tokens=32768,  # Control response length
)

# The prompt template remains the same
prompt = ChatPromptTemplate.from_template(r'''[Contexte] {context}
[Question] {input}

you're an expert at math so solve each question
use mathimathical symbols with their real size (big intégral symbols,
respect fractions size and nominator and dominator sizes correc their sizes )

''')

# Create document chain
document_chain = create_stuff_documents_chain(llm, prompt)

# Create retrieval chain
retriever = db.as_retriever(
    search_kwargs={"k": 6}  # Retrieve top 4 most relevant documents
)

from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever, document_chain)

# Example usage
def query_system(question):
    result = retriever_chain.invoke({'input': question})
    return result['answer']

# Example query
from pprint import pprint

# Example query
query = " donnes des trés trés grandes fractions rationnels et tu doit les simplifier  "
response = query_system(query)

groqrr= response
print( response)
from IPython.display import Markdown, display

from pprint import pprint
display(Markdown(response))

ModuleNotFoundError: No module named 'langchain_groq'

In [6]:
import pathlib
import os
import pytesseract
from PIL import Image # Needed by pytesseract
from pdf2image import convert_from_path # To convert PDF pages to images
# Removed google.generativeai and time imports

# --- Configuration ---
# 🚨 Tesseract Path (MODIFY IF NEEDED) ---
# If Tesseract is not in your system PATH, uncomment and set the correct path below
# On Windows, it might look like: r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# On Linux/macOS, it might be '/usr/bin/tesseract' or '/usr/local/bin/tesseract'
# pytesseract.pytesseract.tesseract_cmd = r'/path/to/your/tesseract'

# 🚨 Poppler Path (MODIFY IF NEEDED) ---
# If Poppler is not in your system PATH, provide the path to its 'bin' directory
# Example for Windows: POPPLER_PATH = r"C:\path\to\poppler-23.08.0\Library\bin"
# Example for macOS/Linux (if installed elsewhere): POPPLER_PATH = "/opt/homebrew/opt/poppler/bin"
POPPLER_PATH = None # Set to None to rely on PATH, or specify the path string

# --- Constants ---
INPUT_PDF_FOLDER = 'LANGAGE Z'  # Folder containing input PDFs
OUTPUT_MD_FOLDER = 'OUTPUT_MD_ALSDD_Zlanguage_OCR_RAW'  # Folder to save output Markdown files
TESSERACT_LANG = 'fra' # Specify French language for Tesseract OCR

# --- Ensure Output Directory Exists ---
pathlib.Path(OUTPUT_MD_FOLDER).mkdir(parents=True, exist_ok=True)

print(f"Starting OCR processing for PDFs in folder: '{INPUT_PDF_FOLDER}'")
print(f"Raw Markdown output will be saved in: '{OUTPUT_MD_FOLDER}'")
print(f"Using Tesseract OCR Language: '{TESSERACT_LANG}'")
print("-" * 30)

# --- Processing Loop ---
pdf_files = [f for f in os.listdir(INPUT_PDF_FOLDER) if f.lower().endswith('.pdf')]

if not pdf_files:
    print(f"No PDF files found in '{INPUT_PDF_FOLDER}'. Exiting.")
else:
    for filename in pdf_files:
        input_filepath = pathlib.Path(INPUT_PDF_FOLDER) / filename
        # Create output filename with .md extension
        if filename.lower().endswith('.pdf'):
            output_filename = filename[:-4] + '.md'
        else:
            output_filename = filename + '.md'
        output_filepath = pathlib.Path(OUTPUT_MD_FOLDER) / output_filename

        print(f"Processing: '{filename}'...")

        extracted_text = ""
        try:
            # 1. Convert PDF to Images
            print(f"  Converting PDF to images...")
            try:
                 # Pass poppler_path if it's set, otherwise pdf2image tries PATH
                 images = convert_from_path(input_filepath, poppler_path=POPPLER_PATH)
            except Exception as convert_err: # Catch specific pdf2image errors if known
                 print(f"  ❌ ERROR: Failed to convert PDF '{filename}' to images.")
                 print(f"      Check if Poppler is installed and accessible via PATH or if POPPLER_PATH is set correctly.")
                 print(f"      Error details: {convert_err}")
                 continue # Skip to next file

            if not images:
                print(f"  ⚠️ WARNING: No images were generated from PDF '{filename}'. It might be empty or corrupted.")
                continue # Skip to next file

            # 2. Perform OCR on each image using Tesseract (French)
            print(f"  Performing OCR with Tesseract (lang='{TESSERACT_LANG}')...")
            full_text_parts = []
            ocr_success = True
            for i, image in enumerate(images):
                try:
                    # Use Tesseract to do OCR on the image with French language
                    page_text = pytesseract.image_to_string(image, lang=TESSERACT_LANG)
                    full_text_parts.append(page_text)
                    print(f"    Processed page {i+1}/{len(images)}")
                except pytesseract.TesseractNotFoundError:
                    print(f"  ❌ CRITICAL ERROR: Tesseract executable not found. Please install Tesseract, add it to PATH, or set pytesseract.tesseract_cmd.")
                    ocr_success = False # Signal critical failure
                    break # Stop processing pages for this file
                except Exception as ocr_err:
                    print(f"  ⚠️ WARNING: OCR failed for page {i+1} of {filename}: {ocr_err}")
                    full_text_parts.append(f"\n\n--- OCR Error on Page {i+1} ---\n\n") # Add error marker

            if not ocr_success:
                 continue # Skip saving for this file due to Tesseract not found

            # Join pages with a clear separator
            extracted_text = "\n\n------ Page Break ------\n\n".join(full_text_parts)

            if not extracted_text.strip():
                 print(f"  ⚠️ WARNING: OCR extracted no text from '{filename}'. Output file will be empty or contain only error markers.")
                 # Still create the file, but it might be empty

            # 3. Save Raw Extracted Text Directly to Markdown File
            print(f"  Saving raw OCR text to Markdown file...")

            # Create simple Markdown content with a header and the raw text in a code block
            output_content = f"# Raw OCR Text from: {filename}\n\n"
            output_content += f"*(Language: {TESSERACT_LANG})*\n\n"
            output_content += "```text\n" # Use a text code block for the raw output
            output_content += extracted_text if extracted_text.strip() else "[No text extracted or only errors occurred]"
            output_content += "\n```\n"

            with open(output_filepath, 'w', encoding='utf-8') as f:
                f.write(output_content)

            print(f"    Successfully extracted text. Raw OCR output saved to: '{output_filepath}'")

        # --- Error Handling ---
        except FileNotFoundError:
            print(f"  ❌ ERROR: Input file not found: '{input_filepath}'")
        except pytesseract.TesseractNotFoundError:
             # This should be caught earlier, but as a safety net
             print(f"  ❌ CRITICAL ERROR: Tesseract is not installed or configured correctly for file '{filename}'. Skipping.")
        except Exception as e:
            # Catch other potential errors (permissions, unexpected Pillow/pdf2image issues)
            print(f"  ❌ UNEXPECTED ERROR processing '{filename}': {type(e).__name__} - {e}")

        print("-" * 30)

    print("Processing finished.")

Starting OCR processing for PDFs in folder: 'LANGAGE Z'
Raw Markdown output will be saved in: 'OUTPUT_MD_ALSDD_Zlanguage_OCR_RAW'
Using Tesseract OCR Language: 'fra'
------------------------------
Processing: 'Khawarizm_.pdf'...
  Converting PDF to images...
  Performing OCR with Tesseract (lang='fra')...
    Processed page 1/100
    Processed page 2/100
    Processed page 3/100
    Processed page 4/100
    Processed page 5/100
    Processed page 6/100
    Processed page 7/100
    Processed page 8/100
    Processed page 9/100
    Processed page 10/100
    Processed page 11/100
    Processed page 12/100
    Processed page 13/100
    Processed page 14/100
    Processed page 15/100
    Processed page 16/100
    Processed page 17/100
    Processed page 18/100
    Processed page 19/100
    Processed page 20/100
    Processed page 21/100
    Processed page 22/100
    Processed page 23/100
    Processed page 24/100
    Processed page 25/100
    Processed page 26/100
    Processed page 27/100


In [30]:
display(Markdown(response.text))

Je vais résoudre chaque question étape par étape.

**Question 1 : Décomposition de la fraction rationnelle F**

On a :

$$F(X) = \frac{1}{(X - 1)^3 (X^2 + 1)^2}$$

La décomposition de F en éléments simples est :

$$F(X) = \frac{a_1}{X - 1} + \frac{a_2}{(X - 1)^2} + \frac{a_3}{(X - 1)^3} + \frac{b_1 X + c_1}{X^2 + 1} + \frac{b_2 X + c_2}{(X^2 + 1)^2}$$

où $a_1, a_2, a_3, b_1, c_1, b_2, c_2$ sont des constantes.

**Calcul des coefficients $a_1, a_2, a_3$**

On effectue le changement d'indéterminée $Y = X - 1$, alors $X = Y+1$ :

$$(X^2 + 1)^2 = ((Y + 1)^2 + 1)^2 = (Y^2 + 2Y + 1 + 1)^2 = (Y^2 + 2Y + 2)^2$$
On a $F(X) = \displaystyle\frac{1}{Y^3 (Y^2+2Y+2)^2}$.
On développe $(Y^2+2Y+2)^2 = 4 + 8Y + 8Y^2 + 4Y^3 + Y^4$.
On cherche la division suivant les puissances croissantes (D.S.P.C.) de $1$ par $4 + 8Y + 8Y^2 + 4Y^3 + Y^4$ à l'ordre 2 :
Soit $1 = (4 + 8Y + 8Y^2 + \dots)(k_0 + k_1 Y + k_2 Y^2 + \dots)$.
$4k_0 = 1 \implies k_0 = \displaystyle\frac{1}{4}$.
$4k_1 + 8k_0 = 0 \implies 4k_1 + 2 = 0 \implies k_1 = -\displaystyle\frac{1}{2}$.
$4k_2 + 8k_1 + 8k_0 = 0 \implies 4k_2 - 4 + 2 = 0 \implies 4k_2 = 2 \implies k_2 = \displaystyle\frac{1}{2}$.
Le quotient est :

$$\frac{1}{4} - \frac{1}{2} Y + \frac{1}{2} Y^2$$

On déduit que $a_3 = k_0, a_2 = k_1, a_1 = k_2$:

$$a_1 = \frac{1}{2}, \quad a_2 = -\frac{1}{2}, \quad a_3 = \frac{1}{4}$$

**Calcul des coefficients $b_2$ et $c_2$**

On pose $H(X) = (X^2+1)^2 F(X) = \displaystyle\frac{1}{(X-1)^3}$.
On évalue $H(X)$ en $X=i$ pour obtenir $b_2 i + c_2$:
$$H(i) = \frac{1}{(i - 1)^3}$$
On a $(i-1)^3 = i^3 - 3i^2(1) + 3i(1)^2 - 1^3 = -i + 3 + 3i - 1 = 2 + 2i$.
Donc,
$$b_2 i + c_2 = \frac{1}{2 + 2i} = \frac{1}{2(1+i)} = \frac{1-i}{2(1+i)(1-i)} = \frac{1-i}{2(1-i^2)} = \frac{1-i}{4}$$
On déduit que :

$$b_2 = -\frac{1}{4}, \quad c_2 = \frac{1}{4}$$

**Calcul des coefficients $b_1$ et $c_1$**

On utilise la méthode des coefficients de Laurent (ou résidus). Soit $F_p(X) = \displaystyle\frac{b_1 X + c_1}{X^2 + 1} + \frac{b_2 X + c_2}{(X^2 + 1)^2}$.
Le coefficient $A_{-2}$ de $(X-i)^{-2}$ dans le développement de $F(X)$ est $\displaystyle\lim_{X \to i} (X-i)^2 F(X) = \lim_{X \to i} \frac{1}{(X-1)^3 (X+i)^2} = \frac{1}{(i-1)^3 (2i)^2} = \frac{1}{(2+2i)(-4)} = \frac{1}{-8-8i} = \frac{-1+i}{16}$.
On a $b_2i+c_2 = -4 A_{-2} = -4 \left(\displaystyle\frac{-1+i}{16}\right) = \displaystyle\frac{1-i}{4}$, ce qui est cohérent.

Le coefficient $A_{-1}$ de $(X-i)^{-1}$ est $\displaystyle\lim_{X \to i} \frac{d}{dX} \left[ (X-i)^2 F(X) \right] = \lim_{X \to i} \frac{d}{dX} \left[ \frac{1}{(X-1)^3 (X+i)^2} \right]$.
La dérivée est $-3(X-1)^{-4}(X+i)^{-2} -2(X-1)^{-3}(X+i)^{-3}$.
En $X=i$:
$A_{-1} = -3(i-1)^{-4}(2i)^{-2} -2(i-1)^{-3}(2i)^{-3}$
$= -3(-4)^{-1}(-4)^{-1} -2(2+2i)^{-1}(-8i)^{-1}$
$= -3\left(\displaystyle\frac{1}{16}\right) -2\left(\displaystyle\frac{1-i}{4}\right)\left(\displaystyle\frac{i}{8}\right) = -\displaystyle\frac{3}{16} - \frac{2(i-i^2)}{32} = -\displaystyle\frac{3}{16} - \frac{i+1}{16} = \frac{-4-i}{16}$.
On a la relation $b_1 i + c_1 = A_{-1}(2i) - \left( \displaystyle\frac{b_2(2i)^2 - (b_2i+c_2)2(2i)}{(2i)^3} \cdot (2i) \right)$ (ceci est $A_{-1}$ pour $F_p(X)$ et non $F(X)$).
Plus simplement, $A_{-1} = \displaystyle\frac{-4b_2 - 4i(b_2i+c_2)}{16} + \frac{b_1i+c_1}{2i}$ (en utilisant la décomposition de $F_p(X)$ en éléments simples sur $\mathbb{C}$).
$\displaystyle\frac{-4-i}{16} = \frac{-4(-1/4) - 4i(1/4-i/4)}{16} + \frac{b_1i+c_1}{2i}$
$\displaystyle\frac{-4-i}{16} = \frac{1 - (i+1)}{16} + \frac{b_1i+c_1}{2i}$
$\displaystyle\frac{-4-i}{16} = \frac{-i}{16} + \frac{b_1i+c_1}{2i}$
$\displaystyle\frac{-4}{16} = \frac{b_1i+c_1}{2i} \implies -\frac{1}{4} = \frac{b_1i+c_1}{2i}$
$b_1i+c_1 = -\displaystyle\frac{1}{4}(2i) = -\frac{i}{2}$.
On déduit que :
$$b_1 = -\frac{1}{2}, \quad c_1 = 0$$

**Décomposition de la fraction rationnelle F**

On a :

$$F(X) = \frac{1/2}{X - 1} - \frac{1/2}{(X - 1)^2} + \frac{1/4}{(X - 1)^3} - \frac{1/2 X}{X^2 + 1} + \frac{-1/4 X + 1/4}{(X^2 + 1)^2}$$

**Question 2 : Décomposition de la fraction rationnelle G**

On a :

$$G(X) = \frac{X}{X^6 - 1}$$

On factorise $X^6-1 = (X-1)(X+1)(X^2+X+1)(X^2-X+1)$.
La décomposition de G en éléments simples est :

$$G(X) = \frac{a_1}{X - 1} + \frac{a_2}{X + 1} + \frac{b_1 X + c_1}{X^2 + X + 1} + \frac{b_2 X + c_2}{X^2 - X + 1}$$

où $a_1, a_2, b_1, c_1, b_2, c_2$ sont des constantes.

**Calcul des coefficients $a_1$ et $a_2$**

Pour $a_1$, on calcule $\displaystyle\lim_{X \to 1} (X-1)G(X) = \lim_{X \to 1} \frac{X(X-1)}{X^6-1}$. En utilisant la règle de L'Hôpital ou $G(X) = P(X)/Q(X) \implies a_1 = P(1)/Q'(1)$:
$P(X)=X, Q(X)=X^6-1 \implies Q'(X)=6X^5$.
$$a_1 = \frac{1}{6(1)^5} = \frac{1}{6}$$
De même, pour $a_2$, on calcule $\displaystyle\lim_{X \to -1} (X+1)G(X) = P(-1)/Q'(-1)$:
$$a_2 = \frac{-1}{6(-1)^5} = \frac{-1}{-6} = \frac{1}{6}$$

**Calcul des coefficients $b_1, c_1, b_2, c_2$**

La fonction $G(X)$ est impaire, i.e. $G(-X)=-G(X)$.
On a $G(-X) = \displaystyle\frac{-a_1}{X+1} + \frac{-a_2}{X-1} + \frac{-b_1X+c_1}{X^2-X+1} + \frac{-b_2X+c_2}{X^2+X+1}$.
Et $-G(X) = \displaystyle-\frac{a_1}{X-1} - \frac{a_2}{X+1} - \frac{b_1X+c_1}{X^2+X+1} - \frac{b_2X+c_2}{X^2-X+1}$.
En comparant les coefficients: $a_1=a_2$ (ce qui est $1/6=1/6$, correct).
Pour les termes en $X^2+X+1$: $-b_2X+c_2 = -(b_1X+c_1) \implies b_2=b_1$ et $c_2=-c_1$.
Pour les termes en $X^2-X+1$: $-b_1X+c_1 = -(b_2X+c_2) \implies b_1=b_2$ et $c_1=-c_2$.
Donc $b_1=b_2$ et $c_1=-c_2$.

Soit $H(X) = G(X) - \displaystyle\frac{1/6}{X-1} - \frac{1/6}{X+1} = \frac{X}{X^6-1} - \frac{1}{6}\frac{X+1+X-1}{(X-1)(X+1)} = \frac{X}{X^6-1} - \frac{X}{3(X^2-1)}$
$H(X) = X \left( \displaystyle\frac{1}{(X^2-1)(X^4+X^2+1)} - \frac{1}{3(X^2-1)} \right) = X \frac{3-(X^4+X^2+1)}{3(X^2-1)(X^4+X^2+1)}$
$H(X) = \displaystyle X \frac{2-X^2-X^4}{3(X^6-1)} = \frac{b_1X+c_1}{X^2+X+1} + \frac{b_2X+c_2}{X^2-X+1}$.
Soit $\omega = e^{i2\pi/3}$ une racine de $X^2+X+1=0$. Alors $\omega^2+\omega+1=0$.
$b_1\omega+c_1 = \displaystyle\lim_{X\to\omega} (X^2+X+1)H(X) = \lim_{X\to\omega} \frac{X(2-X^2-X^4)}{3(X^2-1)(X^2-X+1)}$.
On a $\omega^2=-\omega-1$, $\omega^4=\omega$. Donc $2-\omega^2-\omega^4 = 2-(-\omega-1)-\omega = 3$.
$b_1\omega+c_1 = \displaystyle\frac{\omega(3)}{3(\omega^2-1)(\omega^2-\omega+1)} = \frac{\omega}{(-\omega-2)(-2\omega-2+1)} = \frac{\omega}{(-\omega-2)(-2\omega-1)} = \frac{1}{(2\omega+4)}$.
$b_1\omega+c_1 = \displaystyle\frac{1}{2(-\frac{1}{2}+i\frac{\sqrt{3}}{2})+4} = \frac{1}{-1+i\sqrt{3}+4} = \frac{1}{3+i\sqrt{3}} = \frac{3-i\sqrt{3}}{12} = \frac{1}{4} - i\frac{\sqrt{3}}{12}$.
$b_1(-\frac{1}{2}+i\frac{\sqrt{3}}{2})+c_1 = \frac{1}{4} - i\frac{\sqrt{3}}{12}$.
Par identification des parties réelle et imaginaire :
$b_1\frac{\sqrt{3}}{2} = -\frac{\sqrt{3}}{12} \implies b_1 = -\frac{1}{6}$.
$-\frac{1}{2}b_1+c_1 = \frac{1}{4} \implies -\frac{1}{2}(-\frac{1}{6})+c_1 = \frac{1}{4} \implies \frac{1}{12}+c_1=\frac{3}{12} \implies c_1=\frac{2}{12}=\frac{1}{6}$.
Donc $b_1 = -1/6, c_1 = 1/6$.
Puisque $b_2=b_1$ et $c_2=-c_1$:
$b_2 = -1/6, c_2 = -1/6$.

**Décomposition de la fraction rationnelle G**

On a :

$$G(X) = \frac{1/6}{X - 1} + \frac{1/6}{X + 1} + \frac{-1/6 X + 1/6}{X^2 + X + 1} + \frac{-1/6 X - 1/6}{X^2 - X + 1}$$
$$G(X) = \frac{1}{6} \left( \frac{1}{X - 1} + \frac{1}{X + 1} - \frac{X - 1}{X^2 + X + 1} - \frac{X + 1}{X^2 - X + 1} \right)$$

**Question 3 : Espaces vectoriels de fonctions**

On a $E = \mathcal{A}(\mathbb{R}, \mathbb{R})$ (l'ensemble des fonctions de $\mathbb{R}$ dans $\mathbb{R}$), $F = \{f \in E : f(1) = 0\}$, $G = \{f \in E : f(x) = ax, a \in \mathbb{R}\}$.

**1. Montrer que G est un sous-espace vectoriel de E**

On a :
* La fonction nulle ($f(x)=0 \cdot x = 0$) est dans G (pour $a=0$).
* Si $f_1, f_2 \in G$, alors $f_1(x) = a_1x$ et $f_2(x) = a_2x$ pour $a_1, a_2 \in \mathbb{R}$. Alors $(f_1 + f_2)(x) = f_1(x) + f_2(x) = a_1x + a_2x = (a_1+a_2)x$. Comme $a_1+a_2 \in \mathbb{R}$, $f_1+f_2 \in G$.
* Si $f \in G$ et $\lambda \in \mathbb{R}$, alors $f(x) = ax$ pour $a \in \mathbb{R}$. Alors $(\lambda f)(x) = \lambda (ax) = (\lambda a)x$. Comme $\lambda a \in \mathbb{R}$, $\lambda f \in G$.

On déduit que G est un sous-espace vectoriel de E.

**2. Déterminer F ∩ G**

Soit $f \in F \cap G$. Alors $f \in F$ et $f \in G$.
$f \in G \implies f(x) = ax$ pour un certain $a \in \mathbb{R}$.
$f \in F \implies f(1) = 0$.
Donc $a \cdot 1 = 0 \implies a=0$.
Ainsi $f(x)=0 \cdot x = 0$ pour tout $x \in \mathbb{R}$. Donc $f$ est la fonction nulle.
On a :
$$F \cap G = \{0_E\}$$
où $0_E$ est la fonction nulle.

**3. A-t-on F + G = E ?**

Considérons la fonction $h(x) = 1$. Si $h \in F+G$, alors $h(x) = f(x)+g(x)$ avec $f \in F$ et $g \in G$.
Cela signifie $f(1)=0$ et $g(x)=ax$ pour un $a \in \mathbb{R}$.
Alors $h(1) = f(1)+g(1) = 0 + a \cdot 1 = a$.
Puisque $h(x)=1$ pour tout $x$, $h(1)=1$. Donc $a=1$.
Ceci implique $g(x)=1 \cdot x = x$.
Alors $f(x) = h(x)-g(x) = 1-x$.
Vérifions si $f \in F$: $f(1)=1-1=0$. Oui, $f \in F$.
Donc $h(x)=1$ peut s'écrire $h(x)=(1-x)+x$, où $(1-x) \in F$ et $x \in G$.
Ainsi $h(x)=1 \in F+G$. L'exemple $h(x)=1$ ne montre pas que $F+G \neq E$.

Prouvons que $F+G=E$. Soit $h \in E$ une fonction quelconque.
Cherchons $f \in F$ et $g \in G$ telles que $h=f+g$.
$g \in G \implies g(x)=ax$ pour un $a \in \mathbb{R}$.
$f \in F \implies f(1)=0$.
On doit avoir $h(x)=f(x)+ax$.
En $x=1$: $h(1)=f(1)+a(1) = 0+a = a$.
Donc $a=h(1)$. Ceci détermine $g(x) = h(1) \cdot x$. Cette fonction $g$ est bien dans $G$.
Définissons $f(x) = h(x) - g(x) = h(x) - h(1)x$.
Vérifions si $f \in F$. On calcule $f(1)$:
$f(1) = h(1) - h(1) \cdot 1 = h(1)-h(1)=0$.
Donc $f \in F$.
Ainsi, toute fonction $h \in E$ peut s'écrire comme la somme d'une fonction de $F$ et d'une fonction de $G$.
On a donc $F+G=E$.

**4. En déduire**

En **2**, on a montré que $F \cap G = \{0_E\}$.
En **3**, on a montré que $F+G=E$.
Ces deux conditions signifient que $F$ et $G$ sont des sous-espaces supplémentaires dans $E$.
On écrit alors :
$$E = F \oplus G$$
Si $E$ était de dimension finie, on pourrait utiliser l'argument dimensionnel. L'application $\phi: E \to \mathbb{R}$ définie par $\phi(f)=f(1)$ est une forme linéaire non nulle. $F = \text{Ker}(\phi)$ est donc un hyperplan de $E$. Si $\dim E$ est finie, $\dim F = \dim E - 1$. $G$ est la droite vectorielle engendrée par la fonction $id(x)=x$. Comme $id \notin F$ (car $id(1)=1 \neq 0$), $G$ n'est pas inclus dans $F$.
La formule de Grassmann donne $\dim(F+G) = \dim F + \dim G - \dim(F \cap G)$.
Si $\dim E$ finie: $\dim(F+G) = (\dim E - 1) + 1 - 0 = \dim E$.
Puisque $F+G$ est un sous-espace de $E$ de même dimension que $E$, $F+G=E$.

Je vous remercie de votre patience et de votre attention. J'espère que mes réponses ont été claires et précises. Si vous avez des questions ou des commentaires, n'hésitez pas à me les faire savoir.